In [1]:
import numpy as np
import pandas as pd

# Calculate the Entropy of a dataset
def entropy(data):
    class_counts = data.iloc[:, -1].value_counts()
    probabilities = class_counts / len(data)
    return -np.sum(probabilities * np.log2(probabilities))

# Calculate the Intrinsic Information of a feature
def intrinsic_info(data, feature):
    feature_values = data[feature].value_counts() / len(data)
    return -np.sum(feature_values * np.log2(feature_values))

# Calculate the Gain Ratio of a feature
def gain_ratio(data, feature):
    # Calculate Information Gain
    gain = entropy(data)
    feature_values = data[feature].value_counts()
    weighted_entropy = 0
    for value, count in feature_values.items():
        subset = data[data[feature] == value]
        weighted_entropy += (count / len(data)) * entropy(subset)
    
    information_gain = gain - weighted_entropy
    
    # Calculate Intrinsic Information
    intrinsic = intrinsic_info(data, feature)
    
    # Gain Ratio
    return information_gain / intrinsic if intrinsic != 0 else 0

# Handle continuous features by choosing the best threshold to split
def best_split_continuous(data, feature):
    sorted_data = data.sort_values(by=feature)
    max_gain = 0
    best_threshold = None
    for i in range(1, len(sorted_data)):
        threshold = (sorted_data.iloc[i - 1][feature] + sorted_data.iloc[i][feature]) / 2
        left_subset = data[data[feature] <= threshold]
        right_subset = data[data[feature] > threshold]
        
        gain_left = entropy(left_subset)
        gain_right = entropy(right_subset)
        
        total_entropy = (len(left_subset) / len(data)) * gain_left + (len(right_subset) / len(data)) * gain_right
        gain = entropy(data) - total_entropy
        if gain > max_gain:
            max_gain = gain
            best_threshold = threshold
    
    return best_threshold, max_gain

# C4.5 Algorithm for building the decision tree
def c45(data, features):
    # If all rows have the same class, return the class
    if len(data.iloc[:, -1].unique()) == 1:
        return data.iloc[0, -1]
    
    # If no features left to split on, return the majority class
    if len(features) == 0:
        return data.iloc[:, -1].mode()[0]
    
    # Find the feature with the highest Gain Ratio
    gains = {feature: gain_ratio(data, feature) for feature in features}
    best_feature = max(gains, key=gains.get)
    
    # Create the tree
    tree = {best_feature: {}}
    
    # Handle continuous features
    if isinstance(data[best_feature].iloc[0], (int, float)):
        best_threshold, max_gain = best_split_continuous(data, best_feature)
        tree[best_feature]['<= ' + str(best_threshold)] = c45(data[data[best_feature] <= best_threshold], features)
        tree[best_feature]['> ' + str(best_threshold)] = c45(data[data[best_feature] > best_threshold], features)
    else:
        # Split the data based on the best feature
        feature_values = data[best_feature].unique()
        for value in feature_values:
            subset = data[data[best_feature] == value]
            tree[best_feature][value] = c45(subset, [f for f in features if f != best_feature])
    
    return tree

# Example dataset (replace with your own data)
data = pd.DataFrame({
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Mild', 'Mild', 'Cool', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'High', 'Low', 'Low', 'Low', 'Low', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No']
})

# List of features (excluding the target column)
features = data.columns[:-1].tolist()

# Build the Decision Tree using C4.5
tree = c45(data, features)
print("Decision Tree:")
print(tree)


Decision Tree:
{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Low': 'Yes'}}, 'Overcast': 'Yes', 'Rainy': {'Humidity': {'High': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}, 'Low': 'No'}}}}
